# Data Preprocessing

## Tidying the Data

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.7 MB/s eta 0:00:00


In [ ]:
!pip install nltk emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 8.0 MB/s eta 0:00:00


In [ ]:
# importing the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import emoji
import contractions

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# reading the datasets into dataframes
# reddit files
s1_reddit = pd.read_csv("/content/drive/MyDrive/Umbrella Academy/season 1.csv")
s2_reddit = pd.read_csv("/content/drive/MyDrive/Umbrella Academy/season 2.csv")
s3_reddit = pd.read_csv("/content/drive/MyDrive/Umbrella Academy/season 3.csv")

# youtube files
s1_yt = pd.read_csv("/content/drive/MyDrive/Umbrella Academy/Season 1yt.csv")
s2_yt = pd.read_csv("/content/drive/MyDrive/Umbrella Academy/Season 2yt.csv")
s3_yt = pd.read_csv("/content/drive/MyDrive/Umbrella Academy/Season 3yt.csv")

Checking for missing values in each dataset

In [ ]:
# reddit
print("Null Value Counts for Season 1 Reddit Dataset:")
print(s1_reddit.isnull().sum())
print()  # Add an empty line

print("Null Value Counts for Season 2 Reddit Dataset:")
print(s2_reddit.isnull().sum())
print()  # Add empty line

print("Null Value Counts for Season 3 Reddit Dataset:")
print(s3_reddit.isnull().sum())

Null Value Counts for Season 1 Reddit Dataset:
body          0
word_count    0
dtype: int64

Null Value Counts for Season 2 Reddit Dataset:
body          0
word_count    0
dtype: int64

Null Value Counts for Season 3 Reddit Dataset:
body          0
word_count    0
dtype: int64


In [ ]:
# youtube

print("Null Value Counts for Season 1 YouTube Dataset:")
print(s1_yt.isnull().sum())
print()  # Add an empty line


print("Null Value Counts for Season 2 YouTube Dataset:")
print(s2_yt.isnull().sum())
print()  # Add an empty line


print("Null Value Counts for Season 3 YouTube Dataset:")
print(s3_yt.isnull().sum())

Null Value Counts for Season 1 YouTube Dataset:
author          1
published_at    0
updated_at      0
like_count      0
text            0
dtype: int64

Null Value Counts for Season 2 YouTube Dataset:
author          0
published_at    0
updated_at      0
like_count      0
text            0
dtype: int64

Null Value Counts for Season 3 YouTube Dataset:
author          0
published_at    2
updated_at      2
like_count      2
text            2
dtype: int64


In [ ]:
# we need to drop missing valuse for the s3 youtube data
s3_yt.dropna(inplace=True)

In [ ]:
# Creating a function to add word count column to the youtube datasets
def add_col(df):
  df['word_count'] = df['text'].str.split().str.len()

In [ ]:
# applying the function
add_col(s1_yt)
add_col(s2_yt)
add_col(s3_yt)

Our youtube datasets have columns that will not be necessary for our analysis such as authors name, we will drop them.

In [ ]:
# dropping columns from youtube dataset that are not necessary
# creating a function to do that for all 3 datasets - the function only keeps the text and word count columns
def drop_col(df):
    return df[["text", "word_count"]]

# Applying the function and reassigning the modified DataFrame
s1_yt = drop_col(s1_yt)
s2_yt = drop_col(s2_yt)
s3_yt = drop_col(s3_yt)

To merge the Reddit and YouTube datasets successfully, it is important we standardise the column names representing the comments. In the youtube dataset, the text is labeled as **'text,'** while in the Reddit dataset, it is labeled as **'body.'**

We will harmonise these column names, ensuring that both datasets use either **'text'** or **'body'** as the column name for the textual content. This consistency in column naming will enable us to seamlessly combine the two datasets.

In [ ]:
# Renaming the youtube column 'text' to 'body'
def rename_col(df):
  return df.rename(columns = {"text":"body"})


In [ ]:
# applying function to rename columns
s1_yt = rename_col(s1_yt)
s2_yt = rename_col(s2_yt)
s3_yt = rename_col(s3_yt)

In [ ]:
# We will merge the datasets ie. season 1 reddit with season 1 youtube etc.
# create a function
def concat_df(df1, df2):
    frames = [df1, df2]
    result = pd.concat(frames, ignore_index=True)
    return result

In [ ]:
# apply function to cobine seasins 1 , seasons 2 and seasons 3
s1_df = concat_df(s1_reddit,s1_yt)
s2_df = concat_df(s2_reddit,s2_yt)
s3_df = concat_df(s3_reddit,s3_yt)

In [ ]:
seasons_dataframes = [s1_df, s2_df, s3_df]

for i, season_df in enumerate(seasons_dataframes, start=1):
    print(f"Season {i} DataFrame Information:")
    print(f"Number of Rows: {season_df.shape[0]}")
    print(f"Number of Columns: {season_df.shape[1]}")
    print()  # Add an empty line for better separation

Season 1 DataFrame Information:
Number of Rows: 1310
Number of Columns: 2

Season 2 DataFrame Information:
Number of Rows: 2015
Number of Columns: 2

Season 3 DataFrame Information:
Number of Rows: 2269
Number of Columns: 2



## Applying Natural Language Processing(NLP) techniques

In [ ]:
# defining preprocessing funtion
def preprocess_text(df):
    """
    Preprocesses text data in the 'body' column.

    Args: df: DataFrame with a 'body' column containing text data.

    Returns: DataFrame with preprocessed 'body' column.
    """
    # Initialize stopwords and lemmatizer
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    # Define a function to process each text
    def process_text(text):
        # Lowercase the text
        text = text.lower()

        # Expand contractions
        text = contractions.fix(text)

         # Remove numbers
        text = ''.join([char for char in text if not char.isdigit()])

        # Remove punctuation
        text = ''.join([char for char in text if char not in string.punctuation])

        # Tokenize the text into words
        words = text.split()

        # Remove stopwords
        words = [word for word in words if word not in stop_words]

        # Lemmatize each word
        words = [lemmatizer.lemmatize(word) for word in words]

        # Convert emojis to words
        words = [emoji.demojize(word) for word in words]

        # Rejoin the words into a cleaned text
        cleaned_text = ' '.join(words)

        return cleaned_text

    # Apply the preprocessing function to the 'body' column
    df['body'] = df['body'].apply(process_text)

    return df

In [ ]:
# applying function to season 1 dataframe
s1_df_clean = preprocess_text(s1_df)
s2_df_clean = preprocess_text(s2_df)
s3_df_clean = preprocess_text(s3_df)

In [ ]:
# Creating a function to add new word count column to the  datasets
def add_new_col(df):
    # Calculate word count and add it as a new column
    df['new_word_count'] = df['body'].str.split().str.len()

    # Keep only the 'body' and 'new_word_count' columns
    df = df[['body', 'new_word_count']]

    return df

In [ ]:
# applying new word count column
s1_df_clean = add_new_col(s1_df_clean)
s2_df_clean = add_new_col(s2_df_clean)
s3_df_clean = add_new_col(s3_df_clean)

In [ ]:
# displaying season 1 cleaned dataset
s1_df_clean

,body,new_word_count
0,loved god dang really infuriating luther treat...,12
1,aidan gallagher really deserves prop pretty bi...,10
2,oh boy say fuck peabody guy,6
3,watched episode blown away amazing show probab...,37
4,literally finished series really liked never r...,129
...,...,...
1305,finished reading first vol im pretty stocked w...,13
1306,ign really want u like jacki jing lmao,8
1307,ayy,1
1308,nice like,2


In [ ]:
# displaying season 2 cleaned dataset
s2_df_clean

,body,new_word_count
0,know going lie fence several time season seaso...,19
1,ofcourse diego perfect superhero landing :face...,6
2,need season,2
3,handler shot everyone :eye::mouth::eye: :grinn...,5
4,whole family going road trip feelgood scene,7
...,...,...
2010,waiting,1
2011,loved season much loved different vanya rememb...,8
2012,klaus allisons friendship meant much heartbrok...,10
2013,waiting long review thank keep great work,7


In [ ]:
# displaying season 3 cleaned dataset
s3_df_clean

,body,new_word_count
0,haha love klaus could show lmao,6
1,think agree biggest travesty hotel california ...,8
2,luther see new hot sister :hot_face:,6
3,lady power hallucinogenic gleeking lmao,5
4,thought season much lower budget green screen ...,29
...,...,...
2264,nice,1
2265,awesome video always paul,4
2266,yooo,1
2267,season really amazing :star-struck:,4


We can observe that we have some comments with just 1 word or 2 words. We will remive these comments. We will have a threshhold where we remove comments with less than 5 words.

In [ ]:
# removing comments woth less than 5 words
import pandas as pd

def remove_short_comments(df, min_word_count=5):
    # Filter out comments with less than the specified word count
    filtered_df = df[df['new_word_count'] >= min_word_count]

    return filtered_df

In [ ]:
# applying function
s1_df_clean = remove_short_comments(s1_df_clean,min_word_count=5)
s2_df_clean = remove_short_comments(s2_df_clean,min_word_count=5)
s3_df_clean = remove_short_comments(s3_df_clean,min_word_count=5)

In [ ]:
s1_df_clean

,body,new_word_count
0,loved god dang really infuriating luther treat...,12
1,aidan gallagher really deserves prop pretty bi...,10
2,oh boy say fuck peabody guy,6
3,watched episode blown away amazing show probab...,37
4,literally finished series really liked never r...,129
...,...,...
1297,soon quoti think alone nowquot tiffany sold,7
1298,anyone else think girl play violin one end world,9
1303,im actually looking forward seeing,5
1305,finished reading first vol im pretty stocked w...,13


In [ ]:
s2_df_clean

,body,new_word_count
0,know going lie fence several time season seaso...,19
1,ofcourse diego perfect superhero landing :face...,6
3,handler shot everyone :eye::mouth::eye: :grinn...,5
4,whole family going road trip feelgood scene,7
5,anyone else raise lower volume constantly seem...,14
...,...,...
2004,honestly enjoyed season much got lot ben,7
2009,wish aj bigger part season,5
2011,loved season much loved different vanya rememb...,8
2012,klaus allisons friendship meant much heartbrok...,10


In [ ]:
s3_df_clean

,body,new_word_count
0,haha love klaus could show lmao,6
1,think agree biggest travesty hotel california ...,8
2,luther see new hot sister :hot_face:,6
3,lady power hallucinogenic gleeking lmao,5
4,thought season much lower budget green screen ...,29
...,...,...
2258,amazing video spoiler man :fire::fire:,5
2259,heard rumor mephisto reginald lmao,5
2261,season really great put topping pizza,6
2262,liked seasonbrbrlittle bit le stake since anno...,13


In [ ]:
# convert the clean dataframes to csv files
s1_df_clean.to_csv("season1_clean.csv", index=False)
s2_df_clean.to_csv("season2_clean.csv", index=False)
s3_df_clean.to_csv("season3_clean.csv", index=False)